In [1]:
# Notebook Setup
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Imports
import gym
from gym.wrappers.atari_preprocessing import AtariPreprocessing
from gym.wrappers.frame_stack import FrameStack
from gym.wrappers.monitoring.video_recorder import VideoRecorder

import numpy as np

import tensorflow as tf
from tensorflow import keras

2022-08-07 22:59:49.508028: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/lrae/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/cv2/../../lib64:
2022-08-07 22:59:49.508068: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [10]:
class DQN(keras.Model):
    def __init__(self, num_actions):
        super(DQN,self).__init__()
        
        self.conv1 = keras.layers.Conv2D(
            filters=32,
            kernel_size=8,
            activation='relu',
            input_shape=(84, 84, 4)
        )
        self.pool1 = keras.layers.MaxPool2D(pool_size=(4, 4))
        self.conv2 = keras.layers.Conv2D(
            filters=64,
            kernel_size=4,
            activation='relu'
        )
        self.pool2 = keras.layers.MaxPool2D(pool_size=(2, 2))
        self.conv3 = keras.layers.Conv2D(
            filters=64,
            kernel_size=3,
            activation='relu'
        )
        self.flatten = keras.layers.Flatten()
        self.dense1 = keras.layers.Dense(
            units=512,
            activation='relu'
        )
        self.action = keras.layers.Dense(units=num_actions, activation='linear')
        
    @tf.function
    def call(self, states):
        states = tf.cast(states, tf.float32)
        x = self.conv1(states)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.dense1(x)
        return self.action(x)
        

In [11]:
class DuelingDQN(keras.Model):
    def __init__(self, num_actions):
        super(DuelingDQN, self).__init__()
        
        self.conv1 = keras.layers.Conv2D(
            filters=32,
            kernel_size=8,
            strides=4,
            activation="relu",
            kernel_initializer=keras.initializers.VarianceScaling(2.0),
            data_format="channels_last",
            input_shape=(84,84,4)
        )
        self.conv2 = keras.layers.Conv2D(
            filters=64,
            kernel_size=4,
            strides=2,
            activation="relu",
            kernel_initializer=keras.initializers.VarianceScaling(2.0),
        )
        self.conv3 = keras.layers.Conv2D(
            filters=64,
            kernel_size=3,
            strides=1,
            activation="relu",
            kernel_initializer=keras.initializers.VarianceScaling(2.0),
        )
        self.flatten = keras.layers.Flatten()
        self.dense1 = keras.layers.Dense(
            units=512,
            activation="relu",
            kernel_initializer=keras.initializers.VarianceScaling(2.0),
        )
        self.V = keras.layers.Dense(1)
        self.A = keras.layers.Dense(num_actions)

    @tf.function
    def call(self, states):
        states = tf.cast(states, tf.float32)
        x = self.conv1(states)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.dense1(x)
        V = self.V(x)
        A = self.A(x)
        Q = V + tf.subtract(A, tf.reduce_mean(A, axis=1, keepdims=True))
        return Q

In [5]:
def reward_function(reward):
    return reward

In [3]:
# Create environmnent and preprocess
def instantiate_environmnent():
    env = gym.make("SpaceInvadersNoFrameskip-v4")
    env = AtariPreprocessing(env, grayscale_newaxis=False, frame_skip=5)
    env = FrameStack(env, 4)
    
    return env

In [12]:
%%time

NUM_ACTIONS = 6

# Instantiate environment
env = instantiate_environmnent()

model = DQN(NUM_ACTIONS)
model_target = DQN(NUM_ACTIONS)

# Configuration paramaters for the whole setup
gamma = 0.99 # Discount factor for past rewards
epsilon = 1.0 # Epsilon greedy parameter
epsilon_min = 0.1 # Minimum epsilon greedy parameter
epsilon_max = 1.0 # Maximum epsilon greedy parameter
epsilon_interval = epsilon_max - epsilon_min # Rate at which to reduce chance of random action being taken
batch_size = 32 # Size of batch taken from replay buffer
max_steps_per_episode = 10000

# Optimizer improves training time
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

# Replay buffers
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
score_history = []

# Information variables
running_reward = 0
episode_count = 0
frame_count = 0
explored = 0
exploited = 0

# Number of frames to take random action and observe output and greediness factor
epsilon_random_frames = 150000 # Should change depending on training time
epsilon_greedy_frames = 500000 # Should change depending on training time

# Maximum replay length
max_memory_length = 100000

# Train the model after 4 actions
update_after_actions = 4

# How often to update the target network
update_target_network = 10000

# Using huber loss for stability
loss_function = keras.losses.Huber()


while True: # Run until solved
    state = np.asarray(env.reset()).reshape(84, 84, 4)
    
    # Episode information
    frames_this_episode = 0
    episode_reward = 0
    score = 0

    for timestep in range(1, max_steps_per_episode):
        frame_count += 1
        frames_this_episode += 1

        # Use epsilon-greedy for exploration
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Take random action
            action = np.random.choice(num_actions)
        else:
            # Predict action Q-values from environment state
            state_tensor = tf.convert_to_tensor(state)
            state_tensor = tf.expand_dims(state_tensor, 0)
            action_probs = model(state_tensor, training=False)
            
            # Take best action
            action = tf.argmax(action_probs[0]).numpy()

        # Decay probability of taking random action
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)

        # Apply the sampled action in our environment
        state_next, reward, done, _ = env.step(action)
        state_next = np.asarray(state_next).reshape(84, 84, 4)
        episode_frame_number = _["episode_frame_number"]
        score += reward
        
        # Reward modifier (This will also affect score)
        reward = reward_function(reward)
        
        episode_reward += reward

        # Save actions and states in replay buffer
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next

        # Update every fourth frame and once batch size is over 32
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:

            # Get indices of samples for replay buffers
            indices = np.random.choice(range(len(done_history)), size=batch_size)

            # Using list comprehension to sample from replay buffer
            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array([state_next_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = tf.convert_to_tensor([float(done_history[i]) for i in indices])

            # Build the updated Q-values for the sampled future states
            # Use the target model for stability
            future_rewards = model_target.predict(state_next_sample)
            # Q value = reward + discount factor * expected future reward
            updated_q_values = rewards_sample + gamma * tf.reduce_max(future_rewards, axis=1)

            # If final frame set the last value to -1
            updated_q_values = updated_q_values * (1 - done_sample) - done_sample

            # Create a mask so we only calculate loss on the updated Q-values
            masks = tf.one_hot(action_sample, num_actions)

            with tf.GradientTape() as tape:
                # Train the model on the states and updated Q-values
                q_values = model(state_sample)

                # Apply the masks to the Q-values to get the Q-value for action taken
                q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
                # Calculate loss between new Q-value and old Q-value
                loss = loss_function(updated_q_values, q_action)

            # Backpropagation
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if frame_count % update_target_network == 0:
            # update the the target network with new weights
            model_target.set_weights(model.get_weights())
            # Log details
            print("xxxxxxxxxx")
            template = "running reward: {:.2f} at episode {}, frame count {}"
            print(template.format(running_reward, episode_count, frame_count))

        # Limit the state and reward history
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            print("----------")
            print("----------")
            print(f"Game Number: {episode_count + 1}")
            if frame_count < epsilon_random_frames:
                print("EXPLORATION")
                explored += 1
            else:
                print("EXPLOITATION")
                exploited += 1
            print(f"Score: {score}")
            print(f"Reward: {episode_reward}")
            print(f"Timesteps: {frames_this_episode}")
            print(f"Game Frames Survived: {episode_frame_number}")
            print(f"Epsilon: {epsilon}")
            break

    # Update running reward to check condition for solving
    episode_reward_history.append(episode_reward)
    score_history.append(score)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history[-10:])
    running_score = np.mean(score_history[-10:])
    
    print(f"Running Score (last 10 games): {running_score}")
    print(f"Running Reward (last 10 games): {running_reward}")
    print(f"Explored: {explored}, Exploited: {exploited}")  
    
    # Save model checkpoint
    if episode_count % 100 == 0:
        model.save("model3")
    
    episode_count += 1

    if running_score > 1000:  # Condition to consider the task solved
        print("xxxxxxxxxx")
        print("Solved at episode {}!".format(episode_count))
        break

----------
----------
Game Number: 1
EXPLORATION
Score: 335.0
Reward: 335.0
Timesteps: 680
Game Frames Survived: 3407
Epsilon: 0.9987759999999648
Running Score (last 10 games): 335.0
Running Reward (last 10 games): 335.0
Explored: 1, Exploited: 0


2022-08-07 23:01:13.354973: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: model3/assets
----------
----------
Game Number: 2
EXPLORATION
Score: 130.0
Reward: 130.0
Timesteps: 400
Game Frames Survived: 2005
Epsilon: 0.9980559999999441
Running Score (last 10 games): 232.5
Running Reward (last 10 games): 232.5
Explored: 2, Exploited: 0
----------
----------
Game Number: 3
EXPLORATION
Score: 100.0
Reward: 100.0
Timesteps: 235
Game Frames Survived: 1199
Epsilon: 0.9976329999999319
Running Score (last 10 games): 188.33333333333334
Running Reward (last 10 games): 188.33333333333334
Explored: 3, Exploited: 0


KeyboardInterrupt: 

In [13]:
env = instantiate_environmnent()

model = keras.models.load_model("model2")
video = VideoRecorder(env, "model.mp4", enabled=True)

episodes = 10
for episode in range(1, episodes+1):
    state = np.asarray(env.reset()).reshape(84, 84, 4)
    done = False
    score = 0

    while not done:
        batch_state = tf.expand_dims(state, 0)
        action = np.argmax(model.predict(batch_state)[0])
        state, reward, done, info = env.step(action)
        state = np.asarray(state).reshape(84, 84, 4)
        video.capture_frame()
        score += reward
    
    print('Episode:{} Score:{}'.format(episode, score))

video.close()
env.close()

OSError: No file or directory found at model2